# DC-GAN (Deep Convolutional Gan)

This is an attempt to re-implement the paper DC-GAN

Paper: https://arxiv.org/pdf/1511.06434v2.pdf

In [ ]:
import numpy as np
import tensorflow as tf

In [ ]:
BUFFER_SIZE = 1000
BATCH_SIZE = 256
(train_images, train_labels), (_, _) = tf.keras.datasets.mnist.load_data()

In [ ]:
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')
train_images = (train_images - 127.5) / 127.5  # Normalize the images to [-1, 1]

In [ ]:
# # Batch and shuffle the data
train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
train_dataset_main = tf.data.Dataset.from_tensor_slices(main_data).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [ ]:
train_dataset, train_dataset_main, len(train_dataset)

In [ ]:
def generator(latent_size = 100):
    
    init = tf.random_normal_initializer(0, 0.02)
    
    inp = tf.keras.layers.Input(shape = (latent_size, ))
    
    x = tf.keras.layers.Dense(units = 4 * 4 * 1024, activation = tf.nn.relu, kernel_initializer = init)(inp)
    x = tf.keras.layers.Reshape((4, 4, 1024))(x)
    
    x =  tf.keras.layers.Conv2D(filters = 1024, kernel_size = (3, 3), strides = (1, 1), 
                                  padding = 'same', kernel_initializer = init)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)
    
    x = tf.keras.layers.Conv2DTranspose(filters = 512, kernel_size = (3, 3), strides = (2, 2), 
                                          padding = 'same', kernel_initializer = init)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)
    
    x = tf.keras.layers.Conv2DTranspose(filters = 256, kernel_size = (3, 3), strides = (2, 2), 
                                          padding = 'same', kernel_initializer = init)(x)
    x = tf.keras.layers.ReLU()(x)
    x = tf.keras.layers.BatchNormalization()(x)
    
    x = tf.keras.layers.Conv2DTranspose(filters = 128, kernel_size = (5, 5), strides = (2, 2), 
                                          padding = 'same', kernel_initializer = init)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)
    
    x = tf.keras.layers.Conv2D(filters = 128, kernel_size = (3, 3), strides = (1, 1), 
                                padding = 'valid', kernel_initializer = init)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)
    
    
    x = tf.keras.layers.Conv2D(filters = 1, kernel_size = (3, 3), strides = (1, 1), 
                                padding = 'valid', kernel_initializer = init)(x)
    x = tf.keras.layers.Activation('tanh')(x)
    
    return tf.keras.Model(inputs = inp, outputs = x, name = 'Generator')


def discriminator():
    init = tf.random_normal_initializer(0, 0.02)
    
    inp = tf.keras.layers.Input(shape = (28, 28, 1))
    
    m = tf.keras.layers.Conv2D(filters = 128, kernel_size = (3, 3), strides = (2, 2), 
                                padding = 'same', kernel_initializer = init)(inp)
    m = tf.keras.layers.BatchNormalization()(m)
    m = tf.keras.layers.LeakyReLU(alpha = 0.2)(m)
    
    m = tf.keras.layers.Conv2D(filters = 256, kernel_size = (3, 3), strides = (2, 2), 
                               padding = 'same', kernel_initializer = init)(m)
    m = tf.keras.layers.BatchNormalization()(m)
    m = tf.keras.layers.LeakyReLU(alpha = 0.2)(m)
    
    m = tf.keras.layers.Conv2D(filters = 1, kernel_size = (3, 3), strides = (2, 2), 
                                 padding = 'same', kernel_initializer = init)(m)
    m = tf.keras.layers.BatchNormalization()(m)
    m = tf.keras.layers.LeakyReLU(alpha = 0.2)(m)
    
    f1 = tf.keras.layers.Flatten()(m)
    out = tf.keras.layers.Dense(units = 1, kernel_initializer = init)(f1)
    
    return tf.keras.Model(inputs = inp, outputs = out)

gen = generator()
disc = discriminator()

In [ ]:
loss_function = tf.keras.losses.BinaryCrossentropy(from_logits = True)

def discriminator_loss(disc_real_out, disc_gen_out):
    real_loss = loss_function(tf.ones_like(disc_real_out), disc_real_out)
    gen_loss = loss_function(tf.zeros_like(disc_gen_out), disc_gen_out)
    
    total_loss = real_loss + gen_loss
    return total_loss

def generator_loss(disc_gen_out):
    loss = loss_function(tf.ones_like(disc_gen_out), disc_gen_out)
    return loss

In [ ]:
gen_optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0002, beta_1 = 0.5)
disc_optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0002, beta_1 = 0.5)

In [ ]:
@tf.function
def train_step(real_image):
    latent_space = tf.random.normal([BATCH_SIZE, 100])
    with tf.GradientTape() as disc_tape, tf.GradientTape() as gen_tape:
        gen_out = gen(latent_space, training = True)
        
        disc_real_out = disc(real_image, training = True)
        disc_gen_out = disc(gen_out, training = True)
        
        disc_loss = discriminator_loss(disc_real_out, disc_gen_out)
        gen_loss = generator_loss(disc_gen_out)
        
    disc_gradients = disc_tape.gradient(disc_loss, disc.trainable_variables)
    gen_gradients = gen_tape.gradient(gen_loss, gen.trainable_variables)
    
    disc_optimizer.apply_gradients(zip(disc_gradients, disc.trainable_variables))
    gen_optimizer.apply_gradients(zip(gen_gradients, gen.trainable_variables))
    
    return gen_loss, disc_loss

In [ ]:
def sample_latent_space(size):
    return np.random.normal(0, 2, size = (size, 100))

images = []
def train(dataset, epochs = 1):
    for epoch in range(epochs):
        print(f'Epoch: {epoch}')
        for n, image_batch in enumerate(dataset):
            gen_loss, disc_loss = train_step(image_batch)
            
            if n%10==0: 
                print('.', end = '')
            
        inp = sample_latent_space(1)[0].reshape(1, 100)
        x = gen(inp).numpy()[0]
        images.append(x)
        print (f'\nGenerator:- {gen_loss} Discriminator:- {disc_loss}\n\n\n')

In [ ]:
train(train_dataset, epochs = 1)